In [ ]:
import sys
import valurap
from valurap import rest_client
import time

import imp
imp.reload(rest_client)

c = rest_client.Client()

tgt1 = 230
tgt2 = 230
tgt_hb = 110

c.spp(1, 60000, 0)
c.spp(2, 60000, 0)
c.sfv(ch=1,val=500)
c.sfv(ch=2,val=500)
c.sfv(ch=3,val=1000)
c.spt(ch=1,val=tgt1)
c.spt(ch=2,val=tgt2)
c.spt(ch=3,val=tgt_hb)

print("homing")
c.abort()
c.home()
c.wait_idle()
c.moveto(X1=-190*80, X2=175*80, Y=-16000)
c.wait_idle()

ok1 = 0
ok2 = 0
c.spt(ch=1,val=tgt1)
c.spt(ch=2,val=tgt2)

print("temps approach", tgt1, tgt2)
while True:
    r = c.thermo_state()
    if tgt1 > 0:
        if abs(r["temp1"] - tgt1) < 10 and r["ext1"] > 10 and r["ext1"] < 900:
            ok1 += 1
        else:
            ok1 = 0
    else:
        ok1 = 100
        
    if tgt2 > 0:
        if abs(r["temp2"] - tgt2) < 10 and r["ext2"] > 10 and r["ext2"] < 900:
            ok2 += 1
        else:
            ok2 = 0
    else:
        ok2 = 100
        
    print("{:4d} {:3d} {:2d} | {:4d} {:3d} {:2d} | {:4d} {:3d}".format(
        r["ext1"], r["temp1"], ok1,
        r["ext2"], r["temp2"], ok2,
        r["ext3"], r["temp3"]
    ))
    
    if ok1 > 6 and ok2 > 6:
        break
        
    time.sleep(5)

print("temps tracking", tgt1, tgt2)
c.spp(1, 28000, 400)
c.spp(2, 20000, 350)
ok1 = 0
ok2 = 0
while True:
    r = c.thermo_state()
    if tgt1 > 0:
        if abs(r["temp1"] - tgt1) < 3 and r["ext1"] > 10 and r["ext1"] < 900:
            ok1 += 1
        else:
            ok1 = 0
    else:
        ok1 = 100

    if tgt2 > 0:
        if abs(r["temp2"] - tgt2) < 3 and r["ext2"] > 10 and r["ext2"] < 900:
            ok2 += 1
        else:
            ok2 = 0
    else:
        ok2 = 100
    
    print("{:4d} {:3d} {:2d} | {:4d} {:3d} {:2d} | {:4d} {:3d}".format(
        r["ext1"], r["temp1"], ok1,
        r["ext2"], r["temp2"], ok2,
        r["ext3"], r["temp3"]
    ))

    if ok1 > 23 and ok2 > 23 :
        break
    time.sleep(5)
print("temps ok")
print("prime extruders")

axes = []
if tgt1 > 0:
    axes.append("E1")
if tgt2 > 0:
    axes.append("E2")

c.enable(axes=",".join(axes))

c.move(E1=-50000, E2=-16000)
r = c.wait_idle()
print("all done")


In [ ]:
axes = []
if tgt1 > 0:
    axes.append("E1")
if tgt2 > 0:
    axes.append("E2")

c.enable(axes=",".join(axes))

c.move(E2=-837/2*100)
r = c.wait_idle()
display(r)

In [ ]:
c.abort()
c.sfv(ch=1,val=500)
c.sfv(ch=2,val=500)
c.sfv(ch=3,val=1000)
c.spt(ch=1,val=0)
c.spt(ch=2,val=0)
c.spt(ch=3,val=0)


In [ ]:
imp.reload(rest_client)
c = rest_client.Client()

tgts = [100, 150, 200, 230]
r = c.thermo_state()
c.spp(1,60000, 0)
start_tgt = r["target1"]
for tgt in tgts:
    if tgt < start_tgt:
        continue
    print("approach tgt:", tgt)
    c.spt(ch=1,val=tgt)
    ok = 0
    while True:
        r = c.thermo_state()
        if abs(r["temp1"] - tgt) < 10 and r["ext1"] > 10 and r["ext1"]<900:
            ok += 1
        else:
            ok = 0
        print(r["ext1"],r["temp1"], ok)
        if ok > 3:
            break
        time.sleep(10)

print("tracking")
c.spp(1,20000,400)
ok = 0
while True:
    r = c.thermo_state()
    if abs(r["temp1"] - tgt) < 3 and r["ext1"] > 10 and r["ext1"]<900:
        ok += 1
    else:
        ok = 0
    print(r["ext1"],r["temp1"], ok)
    if ok > 5:
        break
    time.sleep(10)


In [87]:
%matplotlib notebook

import cv2
import math
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
from sklearn.linear_model import LinearRegression
from random import randint
from valurap.optozero import calibration


import numpy as np
x1_mtx = np.array([
    [970.87196948,   0.        , 319.39923978],
    [  0.        , 968.54821206, 222.53847797],
    [  0.        ,   0.        ,   1.        ]
])

x1_dist = np.array([[
    1.62236835e-01,
    -6.54970258e-01,
    3.75880870e-03,
    1.38494135e-03,
    -4.52002591e+00
]])


x2_mtx = np.array([
    [966.18776784,   0.        , 310.95960944],
    [  0.        , 965.62677186, 190.37336236],
    [  0.        ,   0.        ,   1.        ]
])

x2_dist = np.array([[
    1.61754264e-01,
    -6.38459089e-01,
    1.63921796e-03,
    -5.25521344e-03,
    -3.23895623e+00
]])

def pattern_coords(X, Y, axe="X1"):
    if axe == "X1":
        xa = -11715
        ya = -5935
        xb = 6374
        yb = 7278
    else:
        xa = -6477
        ya = 1579
        xb = 11610
        yb = 14778

    x0 = xa + (xb - xa) / 7 * X
    y0 = ya + (yb - ya) / 5 * Y
    
    return x0, y0

def move_to_pattern(X, Y, axe="X1", x_offset=0, y_offset=0):
    x0, y0 = pattern_coords(X,Y,axe)
    c.moveto(**{axe: x0 + x_offset, "Y": y0 + y_offset})
    r = wait(axe)
    return r

def get_circles(attempts=4, axe="X1"):
    if axe == "X1":
        cap = cv2.VideoCapture('http://orange:8081/')
    elif axe == "X2":
        cap = cv2.VideoCapture('http://orange:8082/')
    else:
        raise RuntimeError("Unknown axe", axe)
        
    ret, img = cap.read()
    #ret, img = cap.read()
    #ret, img = cap.read()
    cap.release()
    assert img.shape == (480, 640, 3)

    ret, circles = cv2.findCirclesGrid(img, (6,6))
    if ret:
        cv2.drawChessboardCorners(img, (6, 6), circles, ret)
        #display(circles)
        x = circles[:,0][:,0]
        y = circles[:,0][:,1]
        x0 = np.average(x)
        y0 = np.average(y)
        return x0, y0, img
    else:
        if attempts > 0:
            return get_circles(attempts - 1, axe)
    return (None, None, img)

def get_all_circles(attempts=4, axe="X1"):
    if axe == "X1":
        cap = cv2.VideoCapture('http://orange:8081/')
    elif axe == "X2":
        cap = cv2.VideoCapture('http://orange:8082/')
    else:
        raise RuntimeError("Unknown axe", axe)
        
    ret, img = cap.read()
    #ret, img = cap.read()
    #ret, img = cap.read()
    cap.release()
    assert img.shape == (480, 640, 3)

    ret, circles = cv2.findCirclesGrid(img, (6,6))
    if ret:
        return circles
    else:
        if attempts > 0:
            return get_all_circles(attempts - 1, axe)
    return None

def wait(axe="X1"):
    r = c.wait_idle()
    x = None
    y = None
    for s in r["state"]["apg_states"]:
        if s["name"] == axe:
            x = s["x"]
        if s["name"] == "Y":
            y = s["x"]
            
    return x,y

def get_objp():
    param1 = np.zeros((6 * 6, 3), np.float32)
    param2 = (np.mgrid[-3:3,-3:3] + 0.5).T.reshape(-1,2)
    params = [param1, param2]

    objp = params[0]
    objp[:, :2] = params[1]

    return 2.469438 * objp


def optozero(axe="X1", delta=500):
    a = []
    b = []
    cur_x = 0
    cur_y = 0
    dd = 1
    for i in range(45):
        if i == 20:
            print("optozero: set to 2.5x step")
            dd = 2.5
        elif i == 30:
            print("optozero: set to 5x step")
            dd = 5

        tx = randint(-delta * dd, delta * dd)
        ty = randint(-delta * dd, delta * dd)
        dx = tx - cur_x
        dy = ty - cur_y
        cur_x = tx
        cur_y = ty
        
        c.move(**{axe: dx, "Y": dy})
        r = wait(axe)

        cx, cy, img = get_circles(axe=axe)
        if cx is not None:
            a.append([cx, cy])
            b.append(r)
            if len(a) >= 8:
                break

    if len(a) < 4:
        print("Failed to find target pattern", len(a))
        return None
    
    a = np.array(a)
    b = np.array(b)

    x = b[:,0]
    y = b[:,1]

    reg_x = LinearRegression().fit(a, x)
    reg_y = LinearRegression().fit(a, y)
    tx = reg_x.predict([[320, 240]])[0]
    ty = reg_y.predict([[320, 240]])[0]

    c.moveto(**{axe: int(round(tx)), "Y": int(round(ty))})
    rf = wait(axe)
    return rf


In [ ]:
#c.moveto(X1=1000)
#c.moveto(X2=1000)
base1_x = -4020
base1_y = -9578
base2_x = 1205
base2_y = -2069

c.home()
c.wait_idle()
c.moveto(X1=0)
c.moveto(Y=0)
c.wait_idle()
c.moveto(Z=100000)
c.wait_idle()

c.moveto(X1=base1_x, X2=base2_x, Y=base1_y)
c.wait_idle()
first1_x, first1_y = optozero("X1", delta=50)
display((first1_x, first1_y))

c.moveto(X1=base1_x, X2=base2_x, Y=base2_y)
c.wait_idle()
first2_x, first2_y = optozero("X2", delta=50)
display((first2_x, first2_y))

c.moveto(Z=50000)

for i in range(5):
    c.moveto(X1=first1_x, X2=first2_x, Y=first1_y)
    c.wait_idle()
    r = optozero("X1", delta=50)
    display(r)

    c.moveto(X1=first1_x, X2=first2_x, Y=first2_y)
    c.wait_idle()
    r = optozero("X2", delta=50)
    display(r)


In [ ]:
(-4028.01171875, -9764.01171875)
(1247.9921875, -2236.0078125)
(-4027.01171875, -9765.0078125)
(1242.9921875, -2226.0078125)
(-4026.01171875, -9770.0078125)
(1243.9921875, -2231.0078125)
(-4023.01171875, -9763.01171875)
(1243.98828125, -2227.0078125)
(-4027.01171875, -9764.0078125)
(1247.99609375, -2230.00390625)

(-4000.01171875, -9459.01171875)
(1209.9921875, -1959.01171875)
(-3996.01171875, -9462.0078125)
(1215.9921875, -1934.01171875)
(-3997.01171875, -9466.01171875)
(1213.9921875, -1956.0078125)
(-4000.01171875, -9468.01171875)
(1218.9921875, -1939.0078125)
(-4001.015625, -9474.015625)
(1216.98828125, -1959.0078125)

(-4013.01171875, -9746.01171875)
(1220.9921875, -2207.0078125)
(-4016.0078125, -9748.0078125)
(1219.98828125, -2207.0078125)
(-4016.0078125, -9747.0078125)
(1219.9921875, -2207.0078125)
(-4015.0078125, -9746.0078125)
(1220.9921875, -2208.0078125)
(-4017.0078125, -9748.0078125)
(1220.9921875, -2208.00390625)

(-4029.01171875, -9765.015625)
(1232.9921875, -2225.01171875)
(-4030.015625, -9764.015625)
(1233.9921875, -2226.0078125)
(-4032.01171875, -9765.01171875)
(1232.98828125, -2225.01171875)
(-4032.01171875, -9766.01171875)
(1232.98828125, -2225.01171875)
(-4030.01171875, -9766.0078125)
(1231.98828125, -2225.0078125)

In [ ]:
%matplotlib notebook

import cv2
import math
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
from sklearn.linear_model import LinearRegression
from random import randint




fig=plt.figure(figsize=(8, 8))
rows = 6
columns = 6

axe = "X2"

c.moveto(Z=100000)
c.moveto(X1=0, X2=0, Y=0)
wait(axe)

if axe == "X1":
    c.moveto(X1=0, X2=170*80, Y=0)
else:
    c.moveto(X1=-190*80, X2=0, Y=0)
wait(axe)

x0, y0 = pattern_coords(3, 3, axe)

    
a = []
b = []
for i in range(6):
    x1 = x0 + randint(-500, 500)
    y1 = y0 + randint(-500, 500)

    c.moveto(**{axe: x1, "Y": y1})
    r = wait(axe)

    cx, cy, img = get_circles(axe=axe)
    if cx is not None:
        a.append([cx, cy])
        b.append(r)

    fig.add_subplot(rows, columns, i+1)    
    plt.imshow(img)

a = np.array(a)
b = np.array(b)
display(a)
display(b)

x = b[:,0]
y = b[:,1]

reg_x = LinearRegression().fit(a, x)
reg_y = LinearRegression().fit(a, y)
display(reg_x.coef_)
display(reg_x.intercept_)
display(reg_y.coef_)
display(reg_y.intercept_)
tx = reg_x.predict([[320, 240]])[0]
ty = reg_y.predict([[320, 240]])[0]
display(tx, ty)

c.moveto(**{axe: int(round(tx)), "Y": int(round(ty))})
rf = wait(axe)
xf, yf, img = get_circles(axe=axe)
display(xf, yf)
fig.add_subplot(rows, columns, i+2)    
plt.imshow(img)


In [ ]:
import cv2
import numpy as np


objp = get_objp()
print(objp)

objpoints = []  # 3d point in real world space
imgpoints = []  # 2d points in image plane.
real_state = []  # motion state

axe = "X2"
c.moveto(Z=100000)
c.moveto(X1=0, X2=0, Y=0)
wait(axe)

if axe == "X1":
    xa = -11715
    ya = -5935
    xb = 6374
    yb = 7278
    c.moveto(X1=0, X2=170*80, Y=0)
    wait(axe)

else:
    xa = -6477
    ya = 1579
    xb = 11610
    yb = 14778
    c.moveto(X1=-190*80, X2=0, Y=0)
    wait(axe)

T = (3, 3)
xT0 = xa + (xb - xa) / 7 * T[0]
yT0 = ya + (yb - ya) / 5 * T[1]

T = (1, 1)
xT1 = xa + (xb - xa) / 7 * T[0]
yT1 = ya + (yb - ya) / 5 * T[1]

T = (3, 1)
xT2 = xa + (xb - xa) / 7 * T[0]
yT2 = ya + (yb - ya) / 5 * T[1]

T = (5, 1)
xT3 = xa + (xb - xa) / 7 * T[0]
yT3 = ya + (yb - ya) / 5 * T[1]

for z in range(100000, 19000, -7000):
    c.moveto(Z=z)
    for i in range(10):
        for x0, y0, k in [
            [xT0, yT0, 3],
            [xT1, yT1, 1],
            [xT2, yT2, 1],
            [xT3, yT3, 1],
        ]:
            x1 = x0 + randint(-500*k, 500*k)
            y1 = y0 + randint(-500*k, 500*k)

            c.moveto(**{axe: x1, "Y": y1})
            r = wait(axe)

            circles = get_all_circles(axe=axe)
            if circles is not None:
                objpoints.append(objp)
                imgpoints.append(circles)
                real_state.append(r)



In [ ]:
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img.shape[::-1][1:3], None, None)


In [ ]:
mtx

In [ ]:
dist

In [ ]:
x1_calibration= {
    "objpoints": objpoints,
    "imgpoints": imgpoints,
    "shape": img.shape[::-1][1:3],
    "mtx": mtx,
    "dist": dist,
    "rvecs": rvecs,
    "tvecs": tvecs,
    "real_state": real_state
}


In [ ]:
import pickle
#pickle.dump(x1_calibration, open("x2_calib.pickle", "wb"))

In [ ]:
results = []

for axe in ["X1", "X2"]:
    c.moveto(Z=50000)
    c.moveto(X1=-190*80, X2=170*80, Y=0)

    move_to_pattern(4,3,axe=axe)
    r = optozero(axe, delta=100)
    if r is None:
        print("optozero failed")
        continue

    c.moveto(Z=30000)
    r = optozero(axe, delta=50)
    if r is None:
        print("optozero failed")
        continue

    objp = get_objp()

    a = 150
    for dx, dy in [
        [None, 0],
        [a, a],
        [-2 * a, 0],
        [0, -2 * a],
        [2 * a, 0]
    ]:
        if dx is not None:
            c.move(**{axe: dx, "Y": dy})
            r = wait(axe)

        for z in range(15000, 100000, 1600*5):
            c.moveto(Z=z)
            r = wait(axe)

            for i in range(3):
                circles = get_all_circles(axe=axe)
                if circles is None:
                    print(z, "no circles")
                    continue

                retval, rvec, tvec = cv2.solvePnP(
                    objp, circles,
                    calibration[axe]["mtx"], calibration[axe]["dist"])
                if not retval:
                    print(z, "solvePnP failed")
                    continue

                print(z, retval, list(tvec.T[0]))
                results.append((axe, z, r, rvec, tvec, circles))


In [ ]:
pickle.dump(results, open('bed_leveling_final.pickle', "wb"))

In [ ]:
import pickle 
c1 = pickle.load(open('height_calibration_T4_3.pickle', "rb"))
c2 = pickle.load(open('height_calibration_T4_4.pickle', "rb"))
c3 = pickle.load(open('height_calibration_T5_3.pickle', "rb"))
c4 = pickle.load(open('height_calibration_T5_4.pickle', "rb"))

c_all = c1 + c2 + c3 + c4

In [ ]:
%matplotlib notebook

import pandas as pd
import numpy as np

axe = []
x = []
y = []
z = []
rvec_x = []
rvec_y = []
rvec_z = []
tvec_x = []
tvec_y = []
tvec_z = []
for c in c_all:
    axe_, z_, (x_, y_), rvec, tvec, circles = c
    x_ = int(round(x_))
    y_ = int(round(y_))
    axe.append(axe_)
    x.append(x_)
    y.append(y_)
    z.append(z_)
    rvec_x.append(rvec[0][0])
    rvec_y.append(rvec[1][0])
    rvec_z.append(rvec[2][0])
    tvec_x.append(tvec[0][0])
    tvec_y.append(tvec[1][0])
    tvec_z.append(tvec[2][0])
    

df = pd.DataFrame({"axe": axe, "x": x, "y": y, "z":z, 
                   "rvec_x": rvec_x, "rvec_y": rvec_y, "rvec_z": rvec_z,
                   "tvec_x": tvec_x, "tvec_y": tvec_y, "tvec_z": tvec_z,
                  })

is_x1 = df["x"] * 0
is_x1[df["axe"] == "X1"] = 1.0
df["is_x1"] = is_x1
df["x_c"] = df["x"] / 80.0 + df["tvec_x"] + df["is_x1"] * 70
df["y_c"] = df["y"] / 80.0 + df["tvec_y"] + df["is_x1"] * 95

df["z_c"] = df["z"] / 1600.0 * (1.107 - 0.022 * df["is_x1"]) - df["tvec_z"]

df.plot(x="x", y="z_c", kind='scatter', style="*")

In [ ]:
c.moveto(Z=50000)
c.moveto(X1=-190*80, X2=170*80, Y=0)

objp = get_objp()
results = []
for axe in ["X1", "X2"]:
    for tx in range(8):
        for ty in range(6):
            c.moveto(Z=50000)
            move_to_pattern(tx, ty, axe=axe)
            r = optozero(axe, delta=100)
            if r is None:
                print("optozero failed")
                continue

            c.moveto(Z=30000)
            r = optozero(axe, delta=50)
            if r is None:
                print("optozero failed")
                continue

            a = 100
            cdx = 0
            cdy = 0
            cdz = 0
            for dx, dy, dz in [
                [0, 0, 0],
                [a, a, 0],
                #[-2 * a, 0, 0],
                #[0, -2 * a, 0],
                #[2 * a, 0, 0],
                [0, 0, 1600],
                #[a, a, 0],
                [-2 * a, -2 * a, 0],
                #[0, -2 * a, 0],
                #[2 * a, 0, 0],
            ]:
                cdx += dx
                cdy += dy
                cdz += dz
                
                c.move(**{axe: dx, "Y": dy, "Z": dz})
                r = wait(axe)

                circles = get_all_circles(axe=axe)
                if circles is None:
                    print(tx, ty, cdx, cdy, cdz, "no circles")
                    continue

                retval, rvec, tvec = cv2.solvePnP(
                    objp, circles,
                    calibration[axe]["mtx"], calibration[axe]["dist"])
                if not retval:
                    print(tx, ty, cdx, cdy, cdz, "solvePnP failed")
                    continue

                print(tx, ty, cdx, cdy, cdz, tvec.T[0][2])
                results.append((axe, r, tx, ty, cdx, cdy, cdz, rvec, tvec, circles))


In [ ]:
import pandas as pd
import numpy as np

res = pickle.load(open('bed_leveling_final.pickle', "rb"))

axe = []
x = []
y = []
tx = []
ty = []
cdx = []
cdy = []
cdz = []
rvec_x = []
rvec_y = []
rvec_z = []
tvec_x = []
tvec_y = []
tvec_z = []
for r in res:
    axe_, (x_, y_), tx_, ty_, cdx_, cdy_, cdz_, rvec, tvec, circles = r
    x_ = int(round(x_))
    y_ = int(round(y_))
    axe.append(axe_)
    x.append(x_)
    y.append(y_)
    tx.append(tx_)
    ty.append(ty_)
    cdx.append(cdx_)
    cdy.append(cdy_)
    cdz.append(cdz_)
    rvec_x.append(rvec[0][0])
    rvec_y.append(rvec[1][0])
    rvec_z.append(rvec[2][0])
    tvec_x.append(tvec[0][0])
    tvec_y.append(tvec[1][0])
    tvec_z.append(tvec[2][0])
    

df = pd.DataFrame({"axe": axe,
                   "x": x, "y": y,
                   "tx": tx, "ty": ty,
                   "cdx": cdx, "cdy": cdy, "cdz": cdz,
                   "rvec_x": rvec_x, "rvec_y": rvec_y, "rvec_z": rvec_z,
                   "tvec_x": tvec_x, "tvec_y": tvec_y, "tvec_z": tvec_z,
                  })

df["x_c"] = df["x"] + df["cdx"] + df["tvec_z"]
df["y_c"] = df["y"] + df["cdy"] + df["tvec_z"]
df_x1 = df[
    np.logical_and(
        np.logical_and(df.axe == "X1", df.cdx==0), 
        np.logical_and(df.cdy == 0, df.cdz == 0)
    )
][["tx", "ty", "tvec_z"]].copy()
avg_x1 = df_x1["tvec_z"].mean()
df_x1["dz"] = df_x1.tvec_z - avg_x1

df_x2 = df[
    np.logical_and(
        np.logical_and(df.axe == "X2", df.cdx==0), 
        np.logical_and(df.cdy == 0, df.cdz == 0)
    )
][["tx", "ty", "tvec_z"]].copy()
avg_x2 = df_x2["tvec_z"].mean()
df_x2["dz"] = df_x2.tvec_z - avg_x2

#display(df_x1)
#display(df_x2)
df_m = df_x1.merge(df_x2, on=["tx", "ty"], how="outer")
df_m[["tx", "ty", "dz_x", "dz_y"]]

In [ ]:
import pickle

c.home()
r = c.wait_idle()
print(r)

c.moveto(X1=-190*80, X2=170*80)
r1 = c.wait_idle()
print(r1)

c.moveto(Y=0, Z=10000)
r2 = c.wait_idle()
print(r2)



In [ ]:
from valurap import path_planning2 as pp2
import valurap
from valurap import rest_client
import time
import pickle
import os.path

import imp
imp.reload(rest_client)
imp.reload(pp2)

c = rest_client.Client(emu=False)

class Prn():
    offsets = {
        "E1": {
            "X": 0,
            "Y": 0,
            "Z": 5040 - 1600*(0.8-0.149) #+ 50000
        },
        "E2": {
            "X": (7 - 0.11 - 0.1) * 80,
            "Y": (120 + 2.5 + 0.1)*80,
            "Z": 9110 + 1600*0.066 - 160 #+ 50000
        }
    }
    def __init__(self, c):
        self.c = c
        self.current_X1 = None
        self.current_X2 = None
        self.current_Y = None
        self.current_Z = None
        self.current_extruder = None
        self.planner = pp2.PathPlanner()
        self.accumulated_layers = []
        self.first_send = True
        
    def process_job_end(self):
        sub_chunks = []
        acc_len = 0
        exp_len = 500
        for layer in self.accumulated_layers:
            sub_chunks.append(layer)
            if layer[0] == "segment":
                acc_len += len(layer[2])

            if acc_len > exp_len:
                self.first_send = False
                exp_len = 500
                self.c.exec_code(sub_chunks)
                sub_chunks = []
                acc_len = 0

        if sub_chunks:
            self.c.exec_code(sub_chunks)
            
        print("[PRN] DONE")

    def process_home(self):
        assert not self.accumulated_layers
        print("[PRN] HOME")
        c = self.c
        c.abort()
        time.sleep(2)
        c.home()
        r = c.wait_idle()
        print(r)

        c.moveto(X1=-190*self.planner.spm, X2=170*self.planner.spm)
        rr1 = self.c.wait_idle()
        print(rr1)

        c.moveto(Y=0, Z=20000)
        rr2 = c.wait_idle()
        print(rr2)
        
        prn.set_state(rr1)
        prn.set_state(rr2)
        c.enable(axes="E1,E2")
        c.move(E1=-50000, E2=-16000)
        r = c.wait_idle()


    def process_move(self, x=None, y=None, z=None, e=None):   
        c_e = self.current_extruder
        mapping = {"Y": "Y"}
        segs = []
        
        axe_x = "X1"
        if x is not None:
            real_x = x*self.planner.spm + self.offsets[c_e]["X"]
            if self.current_extruder == "E1":
                dx = real_x - self.current_X1
                self.current_X1 = real_x
            else:
                axe_x = "X2"
                dx = real_x - self.current_X2
                self.current_X2 = real_x

            print("[PRN] MOVE {} TO {} ({})".format(axe_x, real_x, dx))

            segs += self.planner.ext_to_code(dx / self.planner.spm, 100, axe=axe_x)
            
        mapping[axe_x] = "X"

        if y is not None:
            real_y = y*self.planner.spm + self.offsets[c_e]["Y"]
            dy = real_y - self.current_Y
            segs += self.planner.ext_to_code(dy/self.planner.spm, 100, axe="Y")
            print("[PRN] MOVE Y TO {} ({})".format(real_y, dy))
            self.current_Y = real_y

        axe_z = "Z"
        if z is not None:
            real_z = z*self.planner.spmz + self.offsets[c_e]["Z"]
            dz = real_z - self.current_Z
            print("[PRN] MOVE Z TO {} ({}) ()mm".format(real_z, dz, z))
        
            segs += self.planner.ext_to_code(dz/self.planner.spmz, 3, axe="Z")
            self.current_Z = real_z
            assert e is None
        elif e is not None:
            axe_z = c_e
            segs += self.planner.ext_to_code(e, 10, axe=c_e)
            print("[PRN] MOVE {} for {}mm".format(c_e, e))
            
        mapping[axe_z] = "Z"

        tupled_segment = []
        for dt, segs in segs:
            tupled_segment.append((dt, tuple([s.to_tuple() for s in segs])))

        self.accumulated_layers.append(("segment", {
            "map": mapping,
            "acc_step": self.planner.acc_step,
            "extruder": self.current_extruder
        }, tupled_segment))
        
    def process_segment(self, segment, start, end, e):
        assert e == self.current_extruder
        if self.current_extruder == "E1":
            current_x = self.current_X1
        else:
            current_x = self.current_X2
            
        if start:
            print("Expected start", start)
            real_x = start[0]*self.planner.spm + self.offsets[e]["X"]
            real_y = start[1]*self.planner.spm + self.offsets[e]["Y"]
            assert abs(current_x - real_x) < 0.001
            assert abs(self.current_Y - real_y) < 0.001
        print("[PRN] DO SEGMENT", segment[1], len(segment[2]), len(layer_data))
        if end[1][0] is not None:
            real_x = end[1][0]*self.planner.spm + self.offsets[e]["X"]
            real_y = end[1][1]*self.planner.spm + self.offsets[e]["Y"]
            if self.current_extruder == "E1":
                self.current_X1 = real_x
            else:
                self.current_X2 = real_x
            self.current_Y = real_y

            cmd, meta, tup_seg = segment
            seg_len = len(tup_seg)
            print("SEGMENT LENGTH:", seg_len)
            if 1:
                cur_chunk = []
                for step in tup_seg:
                    cur_chunk.append(step)
                    if step == (5, (('X', None, 0), ('Y', None, 0), ('Z', None, 0))):
                        print("  chunk:", len(cur_chunk))
                        self.accumulated_layers.append((cmd, meta, cur_chunk))
                        cur_chunk = []
                if cur_chunk:
                    print("  chunk:", len(cur_chunk))
                    self.accumulated_layers.append((cmd, meta, cur_chunk))                  
            else:
                    self.accumulated_layers.append(segment)

            while True:
                if self.c.emu:
                    break
                r = self.c.state()
                #print(r)
                if r["state"]["code_len"] < 1000000:
                    print("submit code")
                    break
                #print("code_len is too big: {}, sleeping".format(r["state"]["code_len"]))
                time.sleep(1)
                                    
            sub_chunks = []
            acc_len = 0
            
            if self.first_send:
                exp_len = 10000
            else:
                exp_len = 500

            for layer in self.accumulated_layers:
                sub_chunks.append(layer)
                if layer[0] == "segment":
                    acc_len += len(layer[2])
                    
                if acc_len > exp_len:
                    self.first_send = False
                    exp_len = 500
                    self.c.exec_code(sub_chunks)
                    sub_chunks = []
                    acc_len = 0
                    
            self.accumulated_layers = sub_chunks
        else:
            self.accumulated_layers.append(segment)
            print("extruder only segment, just spooling")

    def set_state(self, r):
        if self.c.emu:
            self.current_X1, self.current_X2, self.current_Y, self.current_Z = (
                -15200.4140625, 13598.58984375, -0.41015625, 19951.015625
            )
            return
            
        for st in r["state"]["apg_states"]:
            if st["name"] == "X1":
                self.current_X1 = st["x"]
            if st["name"] == "X2":
                self.current_X2 = st["x"]
            if st["name"] == "Y":
                self.current_Y = st["x"]
            if st["name"] == "Z":
                self.current_Z = st["x"]

        #print(self.current_X1, self.current_X2, self.current_Y, self.current_Z)
        
prn = Prn(c)

fn_tpl = 'helmet_{:05d}.layer'
#fn_tpl = 'box20_void_dual_2_{:05d}.layer'
state = "init"
i = -1
while True:
    i += 1
    fn = fn_tpl.format(i)
    
    while True:
        if os.path.exists(fn):
            break
        print("layer {} is not yet ready".format(fn))
        time.sleep(2)
        
    layer_data = open(fn, 'rb').read()
    layer = pickle.loads(layer_data)
    print("layer:", fn, len(layer))
    start = None
    end = None
    do_home = False
    switch_extruder = None
    segment = None
    
    expected_x = None
    expected_y = None
    expected_z = None
    expected_extruder = None

    for l in layer:
        cmd = l[0]
        print(" cmd:", cmd, state)
        if cmd == "do_home":
            do_home = True
            print("  p:", l)
        elif cmd == "start":
            start = l
            print("  p:", l[3], l[4], l[1])
        elif cmd == "end_state":
            end = l
            print("  p:", l[1])
        elif cmd == "segment":
            assert segment is None
            segment = l
            print("  len: ", len(l[2]))
        elif cmd == "extruder_switch":
            assert segment is None
            switch_extruder = l[1]
            print("  ext: ", l[1])
            
    if state == "init":
        if switch_extruder is not None:
            assert do_home == False and segment is None
            prn.current_extruder = "E{}".format(switch_extruder+1)
        elif do_home:
            prn.process_home()
            if segment:
                assert end[1][0] is None
                assert end[1][1] is None
                prn.process_segment(segment, None, end, prn.current_extruder)
            state = "homed"
        else:
            assert False
        continue
    elif state == "homed":
        assert start[3] == (None, None, 0)
        assert segment
        expected_x = start[1]["X"]
        expected_y = start[1]["Y"]
        expected_z = start[1]["Z"]
        expected_extruder = start[4]
        state = "working"
        prn.process_move(x=expected_x, y=expected_y, z=expected_z)
        prn.process_segment(segment, None, end, expected_extruder)
    elif state == "working":
        if do_home:
            prn.process_job_end()
            break
        elif switch_extruder is not None:
            print("[PRN] SWITCH TO", switch_extruder)
            if prn.current_extruder == "E1":
                target_x = -180
            else:
                target_x = 165
            prn.process_move(x=target_x)
            prn.current_extruder = "E{}".format(switch_extruder+1)
            if segment:
                assert end[1][0] is None
                assert end[1][1] is None
                prn.process_segment(segment, None, end, prn.current_extruder)
            state = "switched"
            print("switched segment:", segment)
            print("switched end", end)
        else:
            assert start[3]
            expected_z = start[1]["Z"]
            print("Z:", expected_z)
            expected_extruder = start[4]
            exp_start = (start[3][0], start[3][1], start[3][2])
            prn.process_move(z=expected_z)
            prn.process_segment(segment, exp_start, end, expected_extruder)
    elif state == "switched":
        print(start[3])
        assert start[3][:2] == (None, None)
        assert segment
        expected_x = start[1]["X"]
        expected_y = start[1]["Y"]
        expected_z = start[1]["Z"]
        expected_extruder = start[4]
        prn.process_move(z=expected_z)
        prn.process_move(y=expected_y)
        prn.process_move(x=expected_x)
        prn.process_segment(segment, None, end, expected_extruder)
        state = "working"
    else:
        assert False
        
while True:
    r = c.state()
    if r["idle"]:
        break
    print(r["state"]["code_len"])
    time.sleep(10)
    
c.abort()
c.home()
c.wait_idle()
c.abort()
if 0:
    c.spt(ch=1, val=0)
    c.spt(ch=2, val=0)
    c.spt(ch=3, val=0)


In [ ]:
if 0:
    while True:
        r = c.state()
        if r["idle"]:
            break
        print(r["state"]["code_len"])
        time.sleep(10)

    c.home()
    c.wait_idle()
c.abort()
c.spt(ch=1, val=0)
c.spt(ch=2, val=0)
c.spt(ch=3, val=0)

In [ ]:
c.abort()
c.home()
c.wait_idle()
c.moveto(Y=0 + 120*80)
c.wait_idle()
c.moveto(X2=0)
c.wait_idle()
#c.move(Z=-3*1600)
#c.wait_idle()

In [ ]:
c.moveto(Z=20000)
r = c.wait_idle()

c.moveto(Z=9110 + 0.0*1600)
r = c.wait_idle()

display(r)

In [ ]:
c.moveto(Z=20000)
r = c.wait_idle()

c.moveto(Z=9110 + 0.3*1600)
r = c.wait_idle()

c.move(X2=-50*80, E2=-837)
r = c.wait_idle()

display(r)

In [ ]:
if 0:
    c.enable(axes="E2")
    c.move(E2=-10000)
    r = c.wait_idle()

#c.move(X2=20*80)
#r = c.wait_idle()
k=-5
k1 = abs(k)
c.move(X2=50*80*k, E2=-837*k1)
r = c.wait_idle()

display(r)

In [60]:
x0 = -30
x1 = 30
y0 = 40
yd = 2
ea = (x1 - x0)*0.3*0.4/(3.14*1.75*1.75/4)
n = 10
s = []
for j in range(2):
    s.append(";---- BEGIN -----")
    s.append("G0 F3600 X{:.2f} Y{:.2f} Z0.3".format(x0 - 10, y0 - 10))
    s.append("G1 F600 E0")
    for i in range(0, n, 2):
        s.append("G0 F3600 X{:.2f} Y{:.2f}".format(x0, y0 + i*yd))
        s.append("G1 F3600 X{:.2f} Y{:.2f} E{:.2f}".format(x1, y0 + i*yd, ea * (i+1)))
        s.append("G0 F3600 X{:.2f} Y{:.2f}".format(x1, y0 + (i+1)*yd))
        s.append("G1 F3600 X{:.2f} Y{:.2f} E{:.2f}".format(x0, y0 + (i+1)*yd, ea * (i+2)))
    s.append("G0 F3600 X{:.2f} Y{:.2f}".format(x0 - 10, y0 + n*yd + 10))
    s.append(";---- END -----")
    s.append("")
    y0 = y0 + yd / 2
    ea = (x1 - x0)*0.3*0.4/(3.14*2.8*2.8/4)
    
print('\n'.join(s))
print('\n'.join(s).replace("X", "@").replace("Y", "X").replace("@", "Y"))

;---- BEGIN -----
G0 F3600 X-40.00 Y30.00 Z0.3
G1 F600 E0
G0 F3600 X-30.00 Y40.00
G1 F3600 X30.00 Y40.00 E2.99
G0 F3600 X30.00 Y42.00
G1 F3600 X-30.00 Y42.00 E5.99
G0 F3600 X-30.00 Y44.00
G1 F3600 X30.00 Y44.00 E8.98
G0 F3600 X30.00 Y46.00
G1 F3600 X-30.00 Y46.00 E11.98
G0 F3600 X-30.00 Y48.00
G1 F3600 X30.00 Y48.00 E14.97
G0 F3600 X30.00 Y50.00
G1 F3600 X-30.00 Y50.00 E17.97
G0 F3600 X-30.00 Y52.00
G1 F3600 X30.00 Y52.00 E20.96
G0 F3600 X30.00 Y54.00
G1 F3600 X-30.00 Y54.00 E23.96
G0 F3600 X-30.00 Y56.00
G1 F3600 X30.00 Y56.00 E26.95
G0 F3600 X30.00 Y58.00
G1 F3600 X-30.00 Y58.00 E29.95
G0 F3600 X-40.00 Y70.00
;---- END -----

;---- BEGIN -----
G0 F3600 X-40.00 Y31.00 Z0.3
G1 F600 E0
G0 F3600 X-30.00 Y41.00
G1 F3600 X30.00 Y41.00 E1.17
G0 F3600 X30.00 Y43.00
G1 F3600 X-30.00 Y43.00 E2.34
G0 F3600 X-30.00 Y45.00
G1 F3600 X30.00 Y45.00 E3.51
G0 F3600 X30.00 Y47.00
G1 F3600 X-30.00 Y47.00 E4.68
G0 F3600 X-30.00 Y49.00
G1 F3600 X30.00 Y49.00 E5.85
G0 F3600 X30.00 Y51.00
G1 F3600 X-30.00 Y

In [81]:
x0 = -60
x1 = 60
y0 = 70
yd = 2
yd_0 = 0
yd_1 = yd / 2

ea = (x1 - x0)*0.3*0.4/(3.14*1.75*1.75/4)
n = 10
ss = []
for j in range(2):
    s = []
    s.append(";---- BEGIN -----")
    s.append("G0 F3600 X{:.2f} Y{:.2f} Z0.3".format(x0 - 10, y0 - 10))
    s.append("G1 F600 E0")
    for i in range(0, n, 2):
        if 1:
            s.append("G0 F3600 X{:.2f} Y{:.2f}".format(x0, y0 + i*yd))
            s.append("G1 F3600 X{:.2f} Y{:.2f} E{:.2f}".format(x1, y0 + i*yd, ea * (i+1)))
            s.append("G0 F3600 X{:.2f} Y{:.2f}".format(x1, y0 + (i+1)*yd))
            s.append("G1 F3600 X{:.2f} Y{:.2f} E{:.2f}".format(x0, y0 + (i+1)*yd, ea * (i+2)))
        else:
            s.append("G0 F3600 X{:.2f} Y{:.2f}".format(x0, y0 + i*yd + yd_0))
            s.append("G1 F3600 X{:.2f} Y{:.2f} E{:.2f}".format(x1, y0 + i*yd + yd_1, ea * (i+1)))
            s.append("G0 F3600 X{:.2f} Y{:.2f}".format(x1, y0 + (i+1)*yd + yd_1))
            s.append("G1 F3600 X{:.2f} Y{:.2f} E{:.2f}".format(x0, y0 + (i+1)*yd + yd_0, ea * (i+2)))
    s.append("G0 F3600 X{:.2f} Y{:.2f}".format(x0 - 10, y0 + n*yd + 10))
    s.append(";---- END -----")
    s.append("")
    y0 = y0 + yd / 2
    ea = (x1 - x0)*0.3*0.4/(3.14*2.8*2.8/4)
    yd_0 = - yd_0
    yd_1 = - yd_1
    ss.append(s)
    
ss = ['\n'.join(s) for s in ss]
ss2 = [s.replace("X", "@").replace("Y", "X").replace("@", "Y").replace(" Z0.3", "") for s in ss]
ss += ss2

s = """
T0
M82 ;absolute extrusion mode
G28 ;Home
G92 E0
G1 F200 E3
G92 E0
G1 F600 E-1

{0}

G92 E0
G1 F600 E-1

{2}

G92 E0
G1 F600 E-1
T1
G92 E0
G1 F300 E-0.5

{1}

G92 E0
G1 F300 E-0.5

{3}

G92 E0
G1 F600 E-1
G0 F3000 X0 Y-200
;Retract the filament
G92 E1
G1 E-1 F300
G28 X0 Y0
""".format(*ss)

print(s)

open("align2.gcode", "w").write(s)


T0
M82 ;absolute extrusion mode
G28 ;Home
G92 E0
G1 F200 E3
G92 E0
G1 F600 E-1

;---- BEGIN -----
G0 F3600 X-70.00 Y60.00 Z0.3
G1 F600 E0
G0 F3600 X-60.00 Y70.00
G1 F3600 X60.00 Y70.00 E5.99
G0 F3600 X60.00 Y72.00
G1 F3600 X-60.00 Y72.00 E11.98
G0 F3600 X-60.00 Y74.00
G1 F3600 X60.00 Y74.00 E17.97
G0 F3600 X60.00 Y76.00
G1 F3600 X-60.00 Y76.00 E23.96
G0 F3600 X-60.00 Y78.00
G1 F3600 X60.00 Y78.00 E29.95
G0 F3600 X60.00 Y80.00
G1 F3600 X-60.00 Y80.00 E35.94
G0 F3600 X-60.00 Y82.00
G1 F3600 X60.00 Y82.00 E41.93
G0 F3600 X60.00 Y84.00
G1 F3600 X-60.00 Y84.00 E47.92
G0 F3600 X-60.00 Y86.00
G1 F3600 X60.00 Y86.00 E53.91
G0 F3600 X60.00 Y88.00
G1 F3600 X-60.00 Y88.00 E59.90
G0 F3600 X-70.00 Y100.00
;---- END -----


G92 E0
G1 F600 E-1

;---- BEGIN -----
G0 F3600 Y-70.00 X60.00
G1 F600 E0
G0 F3600 Y-60.00 X70.00
G1 F3600 Y60.00 X70.00 E5.99
G0 F3600 Y60.00 X72.00
G1 F3600 Y-60.00 X72.00 E11.98
G0 F3600 Y-60.00 X74.00
G1 F3600 Y60.00 X74.00 E17.97
G0 F3600 Y60.00 X76.00
G1 F3600 Y-60.00 X76.0

2799

In [86]:
from valurap import rest_client
c = rest_client.Client()
c.sfv(3,1000)

{'ok': 1}

In [159]:
c.abort()
c.home()
c.wait_idle()

{'idle': True,
 'state': {'apg_states': [{'name': 'X1', 'x': 12698.84765625, 'v': 0},
   {'name': 'X2', 'x': -13501.1484375, 'v': 0},
   {'name': 'Y', 'x': -20001.1484375, 'v': 0}],
  'pc': 51,
  'free_buf': None,
  'code_len': 0,
  'busy': 0},
 'abs_safe': True}

In [162]:
# (-3787.0234375, -9418.0234375)
# (-3791.01171875, -9419.015625)
# (-3790.0234375, -9419.0234375)
#
# (-3771.02734375, -9418.0234375)
# (-3790.01953125, -9417.01953125)
# (-3773.0078125, -9418.0078125)
# (-3789.015625, -9418.015625)
# (-3786.00390625, -9420.00390625)
#
#
# (-3787.0156250, -9423.0156250) (-3808.0156250, -9627.0156250)
# (-3787.0234375, -9422.0234375) (-3808.0234375, -9627.0234375)
# (-3789.0234375, -9425.0234375) (-3810.01953125, -9630.01953125)
# (-3789.015625, -9426.01953125) (-3810.01171875, -9629.0078125)
#
# (-3792.015625, -9420.015625) (-3814.0234375, -9626.01953125)
# (-3794.015625, -9423.01953125) (-3816.015625, -9627.015625)
# (-3796.0078125, -9425.0078125) (-3818.0078125, -9630.01171875)
# (-3798.02734375, -9428.02734375) (-3820.01171875, -9633.015625)
#
#
# (-3780.01171875, -9419.01171875) (-3801.01171875, -9625.01171875)
# (-3781.00390625, -9421.00390625) (-3804.02734375, -9626.02734375)
# (-3781.0, -9421.00390625) (-3805.01171875, -9628.0078125)



c.moveto(Z=100000)
c.wait_idle()
c.moveto(Y=-118*80)
c.wait_idle()
c.moveto(X1=-47*80, X2=18*80)
c.wait_idle()
a = optozero(axe="X1")
c.moveto(Z=50000)
c.wait_idle()
b = optozero(axe="X1")
print(a,b)

(-3796.01953125, -9430.01953125) (-3821.015625, -9636.015625)


In [150]:
# (1477.9921875, -1888.0078125)
# (1471.9765625, -1892.01953125)
# (1471.984375, -1893.015625)
#
# (1472.984375, -1891.015625)
# (1471.9765625, -1890.0234375)
# (1472.984375, -1891.015625)
# (1473.9765625, -1891.0234375)
# (1476.98828125, -1892.01171875)
#
#
# (1475.9765625, -1896.01953125) (1493.98046875, -2078.01953125)
# (1475.9921875, -1896.00781250) (1492.98437500, -2077.01953125)
# (1475.98046875, -1896.01953125) (1492.9921875, -2078.0078125)
# (1474.98828125, -1900.01171875) (1492.99609375, -2080.0078125)
#
# (1476.9765625, -1894.01953125) (1494.984375, -2074.015625)
# (1475.9765625, -1898.0234375) (1492.98828125, -2078.01171875)
# (1473.97265625, -1899.02734375) (1490.98046875, -2080.01953125)
# (1471.98046875, -1903.01953125) (1488.99609375, -2083.0078125)
# (1471.984375, -1902.015625) (1488.984375, -2084.015625)
# (1471.97265625, -1902.0234375) (1487.9765625, -2083.0234375)
# (1470.9921875, -1901.01171875) (1488.98046875, -2084.01953125)
# (1470.984375, -1902.015625) (1488.98046875, -2084.0234375)



c.moveto(Z=100000)
c.moveto(Y=-24*80)
c.wait_idle()
c.moveto(X2=18*80)
c.wait_idle()
a = optozero(axe="X2")
c.moveto(Z=50000)
c.wait_idle()
b = optozero(axe="X2")
print(a,b)


(1470.984375, -1902.015625) (1488.98046875, -2084.0234375)


In [158]:
c.abort()
#c.home()